In [1]:
import torch
import torch.nn as nn
import pprint as pp

In [4]:
import torch

# Assuming you have a tensor of size [16, 1, 12]
tensor = torch.randn(16)
print(tensor.shape)
for i in range(16):
    print(tensor[i])

torch.Size([16])
tensor(1.3575)
tensor(0.0130)
tensor(0.0982)
tensor(-0.0531)
tensor(-0.8901)
tensor(-1.4857)
tensor(-0.0862)
tensor(0.9330)
tensor(2.0888)
tensor(-1.4564)
tensor(1.5044)
tensor(-0.5402)
tensor(-0.6334)
tensor(0.4792)
tensor(-1.4052)
tensor(1.7122)


In [13]:
## testing weight matrix
hidden_size =6
W = nn.Parameter(torch.randn(hidden_size, hidden_size))
timesteps = [1,2,3,4]
clockrates = [1,3,2,5]
def masking_function(hidden_size,clock_val):
    mask = []
    for i in range(hidden_size):
        if i%2==0 or i%clock_val == 0:
            mask.append(1) 
        else : mask.append(0)

    return torch.FloatTensor(mask)


pp.pprint({"Before masking": W})
for i,timestep in enumerate(timesteps):
    mask = masking_function(hidden_size,clockrates[i])
    pp.pprint(W*mask)




{'Before masking': Parameter containing:
tensor([[ 0.1795, -0.2665, -0.5580,  0.8341,  0.1870,  0.2783],
        [ 2.0395, -0.0427,  0.6600, -1.0849, -0.0823, -1.1895],
        [ 1.3630,  1.1436,  0.8650,  0.6246, -0.2393,  0.4165],
        [-0.1855,  1.3889,  0.1607,  1.2954,  1.1938,  0.2196],
        [ 0.4168, -0.1767,  0.4800, -0.3006,  0.9134, -1.0233],
        [-0.3070, -0.3941, -1.1438,  0.5393, -1.4720,  0.2769]],
       requires_grad=True)}
tensor([[ 0.1795, -0.2665, -0.5580,  0.8341,  0.1870,  0.2783],
        [ 2.0395, -0.0427,  0.6600, -1.0849, -0.0823, -1.1895],
        [ 1.3630,  1.1436,  0.8650,  0.6246, -0.2393,  0.4165],
        [-0.1855,  1.3889,  0.1607,  1.2954,  1.1938,  0.2196],
        [ 0.4168, -0.1767,  0.4800, -0.3006,  0.9134, -1.0233],
        [-0.3070, -0.3941, -1.1438,  0.5393, -1.4720,  0.2769]],
       grad_fn=<MulBackward0>)
tensor([[ 0.1795, -0.0000, -0.5580,  0.8341,  0.1870,  0.0000],
        [ 2.0395, -0.0000,  0.6600, -1.0849, -0.0823, -0.0000],
  

In [2]:

import torch
import pprint as pp
def masking_function(hidden_size,clock_val):
    mask = []
    for i in range(hidden_size):
        if i%2==0 or i%clock_val == 0:
            mask.append(1) 
        else : mask.append(0)

    return torch.FloatTensor(mask)

mask = masking_function(12,2)
print(mask[0].dtype)



torch.float32


In [14]:
import random
def generate_sinusoidal_data(num_points, freq=1, amplitude=1):
    t = torch.arange(0, num_points, 1)
    x = amplitude * torch.sin(2 * torch.pi * freq * t / num_points)
    return x
input_sequence = generate_sinusoidal_data(2)
clock_vals = torch.tensor([random.uniform(1, 6) for _ in range(2)])
pp.pprint(input_sequence)
pp.pprint(clock_vals)
input_sequence = torch.tensor([input_sequence,clock_vals])
input_sequence = input_sequence.unsqueeze(1).unsqueeze(1)
print(input_sequence)

tensor([ 0.0000e+00, -8.7423e-08])
tensor([4.0739, 3.0899])


ValueError: only one element tensors can be converted to Python scalars

In [2]:
import random
import numpy as np

def generate_non_repeating_subset(start, end, subset_size):
    if subset_size > (end - start):
        raise ValueError("Subset size cannot be greater than the range size.")

    numbers = list(range(start, end))
    random.shuffle(numbers)
    subset = numbers[:subset_size]
    return subset

import torch

tst = torch.zeros(4)
print(tst[2])

tensor(0.)


In [ ]:
## copy of the torch chive architecture

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pack_sequence, unpack_sequence

class CHIVE(nn.Module):
    def __init__(self, latent_space_dim):
        super(CHIVE, self).__init__()
        self.latent_space_dim = latent_space_dim
        self.encoder = None
        self.model_input = None
        self.frnn_shape = 3
        self.phrnn_shape = 3
        self.sylrnn_shape = 3
        self.input_shape = [self.frnn_shape, self.phrnn_shape, self.sylrnn_shape]

        self._build()

    def _build(self):
        self._build_encoder()

    def _build_encoder(self):
        self.frame_rate_rnn = nn.LSTM(input_size=self.frnn_shape, hidden_size=64, batch_first=True)
        self.phone_rate_rnn= nn.LSTM(input_size=self.phrnn_shape, hidden_size=64, batch_first=True)
        self.syllable_rate_rnn = nn.LSTM(input_size=self.sylrnn_shape, hidden_size=64, batch_first=True)
        # # self.encoder_input = [self.frame_rate_rnn_input, self.phone_rate_rnn_input, self.syllable_rate_rnn_input]

        # # frame_rate_rnn = self.add_rnn_layer(self.frame_rate_rnn_input, self.frnn_shape)
        # # phone_rate_rnn = self.add_rnn_layer(self.phone_rate_rnn_input, self.phrnn_shape)

        # merged_layer = torch.cat([self.frame_rate_rnn[:,-1,:],self.phone_rate_rnn_input, self.syllable_rate_rnn_input], dim=1)
        # syllable_rate_rnn = self.add_rnn_layer(merged_layer.unsqueeze(1), self.sylrnn_shape)

        # bottleneck = self._add_bottleneck(syllable_rate_rnn)

        # self.model_input = self.encoder_input
        # self.encoder = nn.ModuleList([self.frame_rate_rnn_input, self.phone_rate_rnn_input, self.syllable_rate_rnn_input, bottleneck])
        pass

    def summary(self):
        print(self.encoder)

    def compile(self, learning_rate=0.0001):
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.loss_function = nn.MSELoss()
        self.optimizer = optimizer

    def train(self, x_train, y_train, batch_size, num_epochs):
        frnn_train, phrnn_train, sylrnn_train = x_train
        frnn_train = torch.tensor(frnn_train, dtype=torch.float32)
        phrnn_train = torch.tensor(phrnn_train, dtype=torch.float32)
        sylrnn_train = torch.tensor(sylrnn_train, dtype=torch.float32)
        y_train = torch.tensor(y_train, dtype=torch.float32)

        dataset = torch.utils.data.TensorDataset(frnn_train, phrnn_train, sylrnn_train, y_train)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(num_epochs):
            for data in train_loader:
                frnn_batch, phrnn_batch, sylrnn_batch, y_batch = data
                self.optimizer.zero_grad()
                output = self.forward([frnn_batch, phrnn_batch, sylrnn_batch])
                loss = self.loss_function(output, y_batch)
                loss.backward()
                self.optimizer.step()

    def forward(self, x):
        self.frame_rate_rnn, self.phone_rate_rnn, self.syllable_rate_rnn = x
        merged_layer = torch.cat([self.frame_rate_rnn[:, -1, :], self.phone_rate_rnn[:, -1, :], self.syllable_rate_rnn[:, -1, :]], dim=1)
        return self._add_bottleneck(merged_layer.unsqueeze(1))

    def _add_bottleneck(self, x):
        lstm_units = 1
        x, _ = nn.LSTM(input_size=x.size(-1), hidden_size=lstm_units, batch_first=True)(x)
        return x[:, -1, :]

    def add_rnn_layer(self, layer_input, shape):
        lstm_units = 64
        x, _ = nn.LSTM(input_size=shape, hidden_size=lstm_units,batch_first=True)(layer_input)
        return x

if __name__ == "__main__":
    chive = CHIVE(latent_space_dim=1)
    chive.summary()

    num_samples = 1000
    frnn_sequence_length = 2
    phrnn_sequence_length = 3
    sylrnn_sequence_length = 3

    frnn_data = np.random.rand(num_samples, frnn_sequence_length, 2)
    phrnn_data = np.random.rand(num_samples, phrnn_sequence_length, 3)
    sylrnn_data = np.random.rand(num_samples, sylrnn_sequence_length, 3)
    print(frnn_data)
    chive.compile()

    dummy_targets = np.random.rand(800, 1)
    chive.train(x_train=[frnn_data, phrnn_data, sylrnn_data], y_train=dummy_targets, batch_size=16, num_epochs=50)


In [2]:
import random
import numpy as np
import torch
num_samples = 1

test = torch.tensor(np.random.rand(1,1, 13))
print(test[0])

# def generate_non_repeating_subset(start, end, subset_size):
#     if subset_size > (end - start):
#         raise ValueError("Subset size cannot be greater than the range size.")

#     print(start, end, subset_size)
#     numbers = list(range(start, end))
#     random.shuffle(numbers)
#     print(numbers[:8])
#     subset = numbers[:subset_size]
#     return sorted(subset)

# test = generate_non_repeating_subset(0,int(num_samples), int(num_samples/2-1))
# print(test)

tensor([[0.6736, 0.6517, 0.6471, 0.9593, 0.3582, 0.9986, 0.2318, 0.0977, 0.4799,
         0.5009, 0.9110, 0.5143, 0.7781]], dtype=torch.float64)


In [3]:
def average_reduce(input_list, target_length):
    original_length = len(input_list)
    step = original_length // target_length

    reduced_list = [sum(input_list[i:i+step]) / step for i in range(0, original_length, step)]

    return reduced_list[:target_length]

# Example usage
original_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # Your list of 4000 inputs
target_length = 3  # Set the desired length

reduced_list = average_reduce(original_list, target_length)
print(reduced_list)


[2.0, 5.0, 8.0, 3.3333333333333335]


In [8]:
import numpy as np

def average_reduce(input_list, target_length):
    original_length = len(input_list)
    dimension = len(input_list[0])  # Assuming all elements have the same dimensionality
    step = original_length // target_length

    reduced_list = [
        np.mean(input_list[i]) for i in range(0, original_length, step)
    ]

    return reduced_list[:target_length]

# Example usage
original_list = [
    np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    np.array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
        np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    np.array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
        np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    np.array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
        np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    np.array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])]  # Your list of 4000 12-dimensional inputs
target_length = 2  # Set the desired length

reduced_list = average_reduce(original_list, target_length)
print(reduced_list)

[6.5, 6.5]


In [5]:
import numpy as np

def average_reduce(input_list, target_length):
    original_length = len(input_list)
    dimension = len(input_list[0])  # Assuming all elements have the same dimensionality
    step = original_length // target_length

    reduced_list = [
        np.mean(input_list[i:i+step], axis=0) for i in range(0, original_length, step)
    ]

    return reduced_list[:target_length]

# Example usage
original_list = [
    np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    np.array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
        np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    np.array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
        np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    np.array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
        np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
    np.array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])]  # Your list of 4000 12-dimensional inputs
target_length = 2  # Set the desired length

reduced_list = average_reduce(original_list, target_length)
print(reduced_list)

[array([ 7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18.]), array([ 7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18.])]


In [1]:
for i in range(10): print(i)

0
1
2
3
4
5
6
7
8
9


In [2]:
test = [7.521817800171207, 5.804137186790644, 3.963087713555305, 3.4533886165680245, 3.610095469736826, 3.5493848392085625, 3.5721557890435425, 3.6098015516283084, 3.724897184036044]

rc= []

for i in range(len(test)):
    if i==0 : continue
    if i == len(test)-1:break
    rc_val = (test[i] - test[i+1])/(test[i-1] - test[i])
    rc.append(rc_val)
print(rc)    
print(len(test))

[1.0718229331423694, 0.2768524715914172, -0.3074497367075231, -0.3874152872106194, -0.37507351903352437, 1.6532363760661308, 3.0573330039092186]
9


In [3]:
!pip install torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 117.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 31.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 18.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 12.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 9.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
from torchaudio.utils import download_asset

OSError: libcudart.so.12: cannot open shared object file: No such file or directory

In [1]:
SAMPLE_SPEECH = download_asset("../Data_prep/data/aud1.wav")

NameError: name 'download_asset' is not defined